1. Chargement des données

Charger le fichier meta.jsonl en Python.

In [1]:
import json

# Chargement des données
file_path = "meta.jsonl"
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

print(f"Nombre de descriptions chargées : {len(data)}")


Nombre de descriptions chargées : 1000


2. Prétraitement et segmentation des textes
Diviser les descriptions en morceaux.

Solution  : 
Transformation préalable des descriptions qui sont des listes en chaînes de caractères avant de les traiter.

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialisation du Text Splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
product_chunks = []  # Liste pour stocker les morceaux par produit

# Transformation préalable des descriptions en chaînes si elles sont des listes
for item in data:
    if 'description' in item:
        description = item['description']
        if isinstance(description, list):
            # Si c'est une liste, on la convertit en chaîne de caractères
            description = " ".join(description)
        
        # On divise la description convertie en texte
        chunks_for_product = splitter.split_text(description)
        
        # Ajouter les morceaux du produit actuel dans la liste
        product_chunks.append(chunks_for_product)

print(f"Nombre de produits traités : {len(product_chunks)}")
print(f"Nombre total de morceaux générés : {sum(len(chunks) for chunks in product_chunks)}")


Nombre de produits traités : 1000
Nombre total de morceaux générés : 969


3. Création d'un index vectoriel
Générer des embeddings à l’aide de OllamaEmbeddings.
Création de la base de données chrome.
5. Création d’un système de récupération
6. Conception d'une chaîne RAG
7. Exécution de requetes utilisateurs


In [71]:
import os
import json
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import Document
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate

# Charger une partie des données depuis le fichier cleaned_product_descriptions.jsonl
with open('cleaned_product_descriptions.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# Limiter les données à 1/50
data = data[:len(data) // 50]

# Extraire les descriptions
descriptions = [item['description'] for item in data if 'description' in item and item['description']]

# Découper les descriptions en morceaux
chunk_size = 256
chunk_overlap = 64
splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = [splitter.split_text(description) for description in descriptions]

# Aplatir les chunks
flat_chunks = [chunk for sublist in chunks for chunk in sublist]

# Encapsuler les chunks dans des objets Document
documents = [Document(page_content=chunk) for chunk in flat_chunks]

# Utiliser Ollama pour les embeddings
embedding_model = OllamaEmbeddings(model="stablelm2")

# Diviser les chunks en lots
batch_size = 50
batches = [documents[i:i + batch_size] for i in range(0, len(documents), batch_size)]

# Créer les embeddings
embeddings = []
for batch in batches:
    batch_embeddings = embedding_model.embed_documents([doc.page_content for doc in batch])  # Utiliser page_content
    embeddings.extend(batch_embeddings)

# Création de la base de données vectorielle avec Chroma
vectorstore = Chroma.from_documents(documents, embedding_model)

# Configuration du retriever
retriever = vectorstore.as_retriever()

# Utiliser Ollama pour le LLM
llm = Ollama(model="stablelm2")

# Définir le système de prompt
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

# Créer le prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Créer la chaîne de récupération avec le prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

# Exemple de requête
query = "Quel est le meilleur produit pour Samsung Galaxy Note 5 ?"

# Exécuter la chaîne de récupération
response = chain.invoke({"input": query})

# Afficher seulement la réponse générée
print(f"Réponse générée : {response['answer']}")

Réponse générée : Le mieux adapté pour votre smartphone Samsung Galaxy Note 5 est une étui de protection, car il vous permettra d'assurer la sécurité et l'entretien régulier de votre appareil. Les accessoires à clip ou à attaches sont souvent utiles dans ce contexte pour repositionner ou retenir le téléphone sur divers supports tout en conservant les fonctionnalités et performances du smartphone original.


In [ ]:
# la chaîne de récupération
response = chain.invoke({"input": query})

# Afficher la réponse complète pour vérifier sa structure
print(f"Réponse générée : {response}")


Réponse générée : {'input': 'Quel est le meilleur produit pour Samsung Galaxy Note 5 ?', 'context': [Document(metadata={}, page_content='the base; you can reposition and Reattach time and time again. Youll always have the exact grip or Stand you need.'), Document(metadata={}, page_content='the base; you can reposition and Reattach time and time again. Youll always have the exact grip or Stand you need.'), Document(metadata={}, page_content='the base; you can reposition and Reattach time and time again. Youll always have the exact grip or Stand you need.'), Document(metadata={}, page_content='the base; you can reposition and Reattach time and time again. Youll always have the exact grip or Stand you need.')], 'answer': "Afin de choisir le meilleur produit pour votre smartphone Samsung Galaxy Note 5, il convient d'évaluer plusieurs facteurs, tels que la RAM, le stockage et les performances générales. Les options suivantes offrent des configurations optimales pour ce modèle de smartphone 

Définition de RAG :

Une chaîne RAG combine une recherche d'information (retrieval) dans une base de données ou un ensemble de documents 
avec un modèle génératif (LLM) pour produire des réponses précises et contextuelles.

Fonctionnement dans ce code :

Étape 1 : Préparation des données : Les documents sont découpés en morceaux (chunks) et convertis en une base de données vectorielles (Chroma).
Étape 2 : Récupération (retrieval) : Le modèle trouve les documents pertinents pour répondre à une requête utilisateur.
Étape 3 : Génération (Generation) : Les documents récupérés sont utilisés comme contexte pour un modèle de langage (LLM, ici Ollama) 
afin de générer une réponse concise et contextualisée.